In [1]:
import os
import sys
import time
import json
import argparse
from datetime import datetime, timedelta, date
from zoneinfo import ZoneInfo
from urllib.parse import urlencode, urljoin
import urllib.request
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
PRINT_TO_STDOUT = True         # печатать итоговый markdown в вывод ячейки
WEEK_DAYS = 7                  # последняя неделя = 7 дней включая сегодня
UA = "dip-digest-bot/weekly/1.0"
BASE_URL = "https://search.dip.bundestag.de/api/v1/"
SLEEP_SEC = 0.6                # вежливая пауза между страницами

In [ ]:
def api_key() -> str:
    key = os.environ.get("DIP_API_KEY")
    if not key:
        print("Ошибка: установите переменную окружения DIP_API_KEY", file=sys.stderr)
        sys.exit(2)
    return key

def http_get(url: str, headers: dict) -> dict:
    req = urllib.request.Request(url, headers=headers, method="GET")
    with urllib.request.urlopen(req, timeout=60) as resp:
        if resp.status != 200:
            raise RuntimeError(f"HTTP {resp.status} for {url}")
        return json.loads(resp.read().decode("utf-8"))

def build_query(date_start: date, date_end: date, cursor: str | None = None) -> str:
    params = [
        ("f.zuordnung", "BT"),
        ("f.datum.start", date_start.strftime("%Y-%m-%d")),
        ("f.datum.end", date_end.strftime("%Y-%m-%d")),
        ("f.drucksachetyp", "Antwort"),
        ("format", "json"),
    ]
    if cursor:
        params.append(("cursor", cursor))
    return urlencode(params)

def fetch_answers(date_start: date, date_end: date, key: str) -> list[dict]:
    headers = {"Authorization": f"ApiKey {key}", "Accept": "application/json", "User-Agent": UA}
    items = []
    cursor = None
    while True:
        q = build_query(date_start, date_end, cursor)
        url = urljoin(BASE_URL, f"drucksache?{q}")
        data = http_get(url, headers)
        docs = data.get("documents", [])
        items.extend(docs)
        new_cursor = data.get("cursor")
        if not new_cursor or new_cursor == cursor:
            break
        cursor = new_cursor
        time.sleep(SLEEP_SEC)
    return items

def fetch_vorgang(vorgang_id: str, key: str) -> dict:
    headers = {"Authorization": f"ApiKey {key}", "Accept": "application/json", "User-Agent": UA}
    url = urljoin(BASE_URL, f"vorgang/{vorgang_id}?format=json")
    return http_get(url, headers)

def extract_core(doc: dict) -> dict:
    fundstelle = doc.get("fundstelle") or {}
    pdf = fundstelle.get("pdf_url")
    urheber = None
    if isinstance(doc.get("urheber"), list):
        urheber = ", ".join(map(str, doc.get("urheber")))
    elif isinstance(doc.get("urheber"), str):
        urheber = doc.get("urheber")
    elif isinstance(doc.get("autoren"), list):
        urheber = ", ".join(map(str, doc.get("autoren")))
    return {
        "id": doc.get("id"),
        "drucksachetyp": doc.get("drucksachetyp"),
        "dokumentnummer": doc.get("dokumentnummer"),
        "titel": doc.get("titel") or "Без названия",
        "datum": doc.get("datum"),
        "wahlperiode": doc.get("wahlperiode"),
        "urheber": urheber,
        "pdf_url": pdf,
        "vorgangsbezug": doc.get("vorgangsbezug") or [],
    }

def filter_only_ka_ga(items: list[dict], key: str) -> list[dict]:
    result = []
    for d in items:
        entry = extract_core(d)
        vb = entry["vorgangsbezug"]
        keep = False
        types_local = [ref.get("vorgangstyp") for ref in vb if ref.get("vorgangstyp")]
        if any(t in ("Kleine Anfrage", "Große Anfrage", "Grosse Anfrage") for t in types_local):
            keep = True
        elif vb:
            try:
                vg_id = str(vb[0].get("id"))
                vg = fetch_vorgang(vg_id, key)
                vt = vg.get("vorgangstyp")
                if vt in ("Kleine Anfrage", "Große Anfrage", "Grosse Anfrage"):
                    keep = True
            except Exception:
                keep = False
        if keep:
            result.append(entry)
    return result

def group_key(urheber: str | None) -> str:
    if not urheber:
        return "Unbekannt"
    return urheber.strip() or "Unbekannt"

def build_md(date_start: date, date_end: date, entries: list[dict]) -> str:
    head = (
        f"# Antworten der Bundesregierung auf Kleine/Große Anfragen\n"
        f"## Zeitraum: {date_start.strftime('%Y-%m-%d')} – {date_end.strftime('%Y-%m-%d')}\n\n"
    )
    if not entries:
        return head + "_Ничего не найдено._\n"

    entries_sorted = sorted(entries, key=lambda e: (group_key(e["urheber"]), e.get("datum") or "", e.get("dokumentnummer") or ""))
    md = [head]
    current = None
    for e in entries_sorted:
        g = group_key(e["urheber"])
        if g != current:
            md.append(f"## {g}\n")
            current = g
        line = f"- **{e['titel']}**"
        if e.get("dokumentnummer"):
            line += f" · BT-Drucksache {e['dokumentnummer']}"
        if e.get("drucksachetyp"):
            line += f" · {e['drucksachetyp']}"
        if e.get("datum"):
            line += f" · {e['datum']}"
        if e.get("pdf_url"):
            line += f" · [PDF]({e['pdf_url']})"
        md.append(line)
    total = len(entries)
    md.append("")
    md.append(f"_Всего ответов: {total}._\n")
    return "\n".join(md)

In [4]:
tz = ZoneInfo("Europe/Berlin")
today = datetime.now(tz).date()
week_start = today - timedelta(days=WEEK_DAYS - 1)
key = api_key()

print(f"Загружаем ответы с {week_start} по {today}...")
raw = fetch_answers(week_start, today, key)
filtered = filter_only_ka_ga(raw, key)
md = build_md(week_start, today, filtered)

out_name = f"digest-answers-week-{today.strftime('%Y%m%d')}.md"
with open(out_name, "w", encoding="utf-8") as f:
    f.write(md)

if PRINT_TO_STDOUT:
    print(md)

print(f"Готово. Файл: {out_name}. Ответов: {len(filtered)}")

Загружаем ответы с 2025-10-13 по 2025-10-19...
# Antworten der Bundesregierung auf Kleine/Große Anfragen
## Zeitraum: 2025-10-13 – 2025-10-19

_Ничего не найдено._

Готово. Файл: digest-answers-week-20251019.md. Ответов: 0
